In [27]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session

In [28]:
import pandas as pd
s3 = boto3.client('s3')

In [29]:
# set an output path where the trained model will be saved
bucket_name = 'n2bucket'
prefix = 'xgboost-built-in'

output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://n2bucket/xgboost-built-in/output


In [30]:
#downloading the file from web to session
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "cleaned_bank_data.csv")
    print('Success: downloaded cleaned_bank_data.csv. file in the session')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded cleaned_bank_data.csv. file in the session


In [31]:
#loading the data from session to the notebook
try:
    bank_df = pd.read_csv('./cleaned_bank_data.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: Data loaded into dataframe.


In [32]:
bank_df.head()

,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,56,1,999,0,1,0,0,0,0,1,...,0,1,0,0,0,0,1,0,1,0
1,57,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
2,37,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
3,40,1,999,0,1,0,1,0,0,0,...,0,1,0,0,0,0,1,0,1,0
4,56,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0


In [33]:
bank_df.columns

Index(['age', 'campaign', 'pdays', 'previous', 'no_previous_contact',
       'not_working', 'job_admin.', 'job_blue-collar', 'job_entrepreneur',
       'job_housemaid', 'job_management', 'job_retired', 'job_self-employed',
       'job_services', 'job_student', 'job_technician', 'job_unemployed',
       'job_unknown', 'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'default_no', 'default_unknown', 'default_yes',
       'housing_no', 'housing_unknown', 'housing_yes', 'loan_no',
       'loan_unknown', 'loan_yes', 'contact_cellular', 'contact_telephone',
       'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep',
       'day_of_week_fri', 'day_of_week_mon'

In [34]:
# Train Test split 

import warnings
warnings.filterwarnings("ignore")

import numpy as np
train_data, test_data = np.split(bank_df.sample(frac=1, random_state=1729), [int(0.7 * len(bank_df))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [35]:
train_data

,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
40949,54,3,999,0,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
9332,56,2,999,0,1,0,0,1,0,0,...,1,0,0,0,0,0,1,0,1,0
32286,32,2,999,0,1,0,0,1,0,0,...,1,0,0,0,0,0,1,0,1,0
3925,46,3,999,0,1,0,1,0,0,0,...,0,1,0,0,0,0,1,0,1,0
9406,35,2,999,0,1,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3871,32,2,999,0,1,0,0,1,0,0,...,1,0,0,0,0,0,1,0,1,0
16681,26,2,999,0,1,0,0,1,0,0,...,0,0,0,0,1,0,1,0,1,0
39272,33,1,6,2,0,0,1,0,0,0,...,0,1,0,0,0,0,0,1,0,1
7717,32,4,999,0,1,0,1,0,0,0,...,1,0,0,0,0,0,1,0,0,1


In [36]:
# Saving Train And Test Into Buckets
# Let's start with Train Data
import os
import sagemaker
import boto3


pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'],
                                                axis=1)],
                                                axis=1).to_csv('train.csv', index=False, header=False)

boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')

s3_input_train = s3_input(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [37]:
# Test Data Into Buckets
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)

boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')

s3_input_test = s3_input(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


# Retrieving  Xgboot docker image (Built-in)


In [38]:
# docker image = module image in a container
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# URI is used to identify and locate resources on the internet

import sagemaker

container = sagemaker.image_uris.retrieve(
    framework='xgboost',
    region=boto3.Session().region_name,
    version='1.0-1'
)

print(container)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker.image_uris:Defaulting to only supported image scope: cpu.


257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3


In [39]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [40]:
import sagemaker

# Construct a SageMaker estimator for XGBoost
estimator = sagemaker.estimator.Estimator(
    image_uri=container,  # Container image URI for XGBoost
    hyperparameters=hyperparameters,  # Hyperparameters for XGBoost
    role=sagemaker.get_execution_role(),  # IAM role for SageMaker
    instance_count=1,  # Number of instances for training
    instance_type='ml.m5.2xlarge',  # Instance type for training
    volume_size=5,  # EBS volume size for training data
    output_path=output_path,  # S3 location for model artifacts
    use_spot_instances=True,  # Whether to use Spot instances for training
    max_run=300,  # Maximum training time
    max_wait=600  # Maximum waiting time for Spot instances
)

In [41]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-03-29-21-20-51-931


2024-03-29 21:20:52 Starting - Starting the training job...
2024-03-29 21:21:06 Starting - Preparing the instances for training......
2024-03-29 21:21:59 Downloading - Downloading input data...
2024-03-29 21:22:40 Training - Training image download completed. Training in progress...[2024-03-29 21:23:01.309 ip-10-0-201-12.us-east-2.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Single node tr

# Deploying Xgboost Model As Endpoints

In [42]:
xgb_predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium'
)

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-03-29-21-26-19-824
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-03-29-21-26-19-824
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-03-29-21-26-19-824


--------!

# Prediction based on the Test Data:

In [43]:
# Serializing input data into CSV format means converting the input data into a comma-separated values (CSV) format. 

# Serilization is needed when we need to work on different system with different environment.....

from sagemaker.serializers import CSVSerializer

test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values

xgb_predictor.content_type = 'text/csv'  # Set the data type for inference
xgb_predictor.serializer = CSVSerializer()  # Set the serializer type

predictions = xgb_predictor.predict(test_data_array).decode('utf-8')  # Predict

predictions_array = np.fromstring(predictions[1:], sep=',')  # Convert predictions into an array

print(predictions_array.shape)

(12357,)


In [44]:
predictions_array

array([0.05214286, 0.05660191, 0.05096195, ..., 0.03436061, 0.02942475,
       0.03715819])

In [48]:
# Set the threshold
threshold = 0.5

# Convert probabilities to binary predictions based on the threshold
binary_predictions = ['yes' if p > threshold else 'no' for p in predictions_array]

print(binary_predictions)

['no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'n

In [45]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100

print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 



In [46]:
# End point: Deployed model which can predict a single output in terms of yes/no.
#Delete end point
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2024-03-29-21-26-19-824


In [47]:
# Delete object from s3 bucket
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': '2TTHVP37RDD24JG4',
   'HostId': 'b0OJCayyehZegD0o66FmmtBGs+8RaI19ZgbHTYteuiSMg0IzdIgpOMuxApvPt84JwgYcSl4g6PJRRn0qWGOMMA==',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'b0OJCayyehZegD0o66FmmtBGs+8RaI19ZgbHTYteuiSMg0IzdIgpOMuxApvPt84JwgYcSl4g6PJRRn0qWGOMMA==',
    'x-amz-request-id': '2TTHVP37RDD24JG4',
    'date': 'Fri, 29 Mar 2024 22:06:30 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-built-in/output/sagemaker-xgboost-2024-03-29-21-12-36-191/profiler-output/system/incremental/2024032921/1711746840.algo-1.json',
    'DeleteMarker': True,
    'DeleteMarkerVersionId': 'fE7gBTwJuxC_J0nkuP_E3FU6WY7.4r5K'},
   {'Key': 'xgboost-built-in/output/sagemaker-xgboost-2024-03-29-21-20-51-931/debug-output/events/000000000030/000000000030_worker_0.tfevents',
    'DeleteMarker': True,
    'DeleteMarkerVersion

We can still make an endpoint solution to arrive at the final decision for a loan: yes/no, accept/decline. This can be implemented even by the non-techninal team member of a company to make final dicision of a business problem.